# Agent Access Control with OAuth, MCP, and RAG

This notebook models how an AI agent safely accesses
tools and documents using OAuth scopes.

Goal:
- Treat agents as OAuth clients
- Enforce least privilege
- Prevent unintended or unsafe actions


## Agents Are OAuth Clients

An AI agent is NOT special from a security perspective.

It is:
- An OAuth client
- With a client_id
- With scopes
- With bounded authority

Treating agents as “special” bypasses security.


## Agent Identity Recap (CIMD)

Agent identity:
- client_id = HTTPS URL
- Metadata fetched by Authorization Server
- Cached and revocable

Identity answers:
"Who is asking?"
Authorization answers:
"What may they do?"

## End-to-End Agent Flow

1. Agent initiates OAuth flow
2. Authorization Server resolves agent identity (CIMD)
3. Authorization Server issues scoped token
4. Agent calls MCP tools or APIs
5. Resource Servers enforce scopes
6. Agent generates output using allowed data only



---

## 🧰 Cell 4 — MCP tools as Resource Servers (Markdown)

```markdown
## MCP Tools as Resource Servers

Each MCP tool is effectively a Resource Server.

Examples:
- search_papers
- download_pdf
- summarize_document

Each tool:
- Receives a token
- Checks required scope
- Executes or rejects


## Scope to Tool Mapping

Example:

- scope: research.search
  → tool: search_papers

- scope: research.download
  → tool: download_pdf

- scope: research.summarize
  → tool: summarize_document

If the token lacks scope:
→ tool MUST NOT execute



---

## 📚 Cell 6 — RAG document access control (Markdown)

```markdown
## RAG Access Control Using OAuth Scopes

RAG systems must enforce authorization
BEFORE retrieval, not after generation.

Example scopes:
- doc.read:cardiology
- doc.read:oncology



---

## 🔍 Cell 7 — Example: RAG retrieval gate (Code)

```python
def retrieve_documents(token_data, topic):
    required_scope = f"doc.read:{topic}"
    scopes = token_data["scope"].split()

    assert required_scope in scopes, "document access denied"

    return f"documents for {topic}"



---

## 🚨 Cell 8 — Why hallucinations don’t bypass OAuth (Markdown)

```markdown
## Why Hallucinations Don’t Bypass Authorization

AI models can hallucinate text.
They cannot hallucinate access.

Authorization failures occur:
- Before retrieval
- Before tool execution
- Outside the model

Security failures come from bad scope design,
not model behavior.


## Dangerous Agent Anti-Patterns

❌ Give agents wildcard scopes
❌ Let prompts decide access
❌ Skip OAuth for “internal” agents
❌ Merge identity and authorization
❌ Let models call tools directly


## Agent Authorization Summary

- Agents are OAuth clients
- CIMD establishes identity
- Scopes define authority
- MCP tools enforce scopes
- RAG retrieval enforces scopes
- Generation is always downstream
